In [2]:
import pandas as pd
import streamlit as st
from collections import Counter

# Обзор данных

In [62]:
st.set_page_config(page_title="Количество задач", layout="wide")


uploaded_file = st.file_uploader("Выбирете файл")


if uploaded_file is not None:
     df = pd.read_csv(uploaded_file, sep='|')
     df.columns = [
        'task_id',
        'task_name',
        'status',
        'communication_method',
        'date_creation',
        'customer_phone',
        'plathorm']

     # изменение типов
     df['customer_phone'] = df['customer_phone'].astype(str)
     df['task_id'] = df['task_id'].astype(int)
     df['date_creation'] = pd.to_datetime(df['date_creation'], format='%Y-%m-%dT%H:%M:%S')

     # удвление тестовых номеров, которые начинаются на 520...
     df['phone'] = df.customer_phone.str[:3]
     df = df[~df.phone.str.contains('520')]

     # удаление пропусков, которые появляются из за особенностей выгрузки
     df = df.dropna()
     file_container = st.expander("Check your uploaded .csv")   
     st.write(df)
else:
    st.info(
        f"""
             👆 Загрузите файл с расширением .csv
             
             В файле должны стого содержаться следующие столбцы: id задачи, название задачи, текущий статус задачи, 
             выбранный способ связи в приложении, дата создания задачи в формате ГГГГ-ММ-ДД, телефон заказчика
             """
    )
    st.stop()

StopException: 

In [3]:
st.write("""
**Поля датафрейма:**
- task_id - уникальный идентификатор задачи;
- task_name - название задачи;
- status - текущий статус задачи;
- communication_method - выбранный способ связи в приложении;
- date_creation - дата создания задачи в формате ГГГГ-ММ-ДД
- customer_phone - телефон заказчика""")

2022-04-25 12:10:37.818 
  command:

    streamlit run /Users/arturfattahov/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
# df = pd.read_csv('/Users/arturfattahov/Downloads/task_september_february.csv', sep='|')

# df.columns = [
#         'task_id',
#         'task_name',
#         'status',
#         'communication_method',
#         'date_creation',
#         'customer_phone',
#         'plathorm'
# ]

# # изменение типов
# df['customer_phone'] = df['customer_phone'].astype(str)
# df['task_id'] = df['task_id'].astype(int)
# df['date_creation'] = pd.to_datetime(df['date_creation'], format='%Y-%m-%dT%H:%M:%S')

# # удвление тестовых номеров, которые начинаются на 520...
# df['phone'] = df.customer_phone.str[:3]
# df = df[~df.phone.str.contains('520')]

# # удаление пропусков, которые появляются из за особенностей выгрузки
# df = df.dropna()

# df.info()
# df.sample(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72775 entries, 0 to 72870
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   task_id               72775 non-null  int64         
 1   task_name             72775 non-null  object        
 2   status                72775 non-null  object        
 3   communication_method  72775 non-null  object        
 4   date_creation         72775 non-null  datetime64[ns]
 5   customer_phone        72775 non-null  object        
 6   plathorm              72775 non-null  object        
 7   phone                 72775 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 5.0+ MB


,task_id,task_name,status,communication_method,date_creation,customer_phone,plathorm,phone
4363,113018,me puede cotizar porfavor una mudanza de león ...,selection,через телефон,2022-01-24 20:29:19.388354,524774456575,admins,524
34807,82292,Busco renta de mesas de este tipo con sus sill...,archived,через телефон,2021-11-23 19:58:24.839278,524777000052,admins,524


In [5]:
df = df.sort_values(by=['customer_phone', 'date_creation'])

In [6]:
df['diff'] = df.groupby('customer_phone')['date_creation'].diff()

# Фильтры по платформам

In [7]:
# df["days"] = df["date_creation"].astype(str).str[0:10]

In [8]:
# df = df[(df['days'] < '2022-01-28')]

In [9]:
df_admins = df[df['plathorm'] == 'admins']
df_ios = df[df['plathorm'] == 'ios']
df_andoid = df[df['plathorm'] == 'android']

# Рассчеты

In [17]:
st.write("""# Рассчеты""")

In [18]:
st.write("""## Admins""")

In [19]:
unique_users_admins = df_admins['customer_phone'].nunique()
number_tasks_admins = df_admins['task_id'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну задачу:", unique_users_admins)
st.write("Количество созданных задач:", number_tasks_admins)

In [20]:
st.write("""## iOS""")

In [21]:
unique_users_ios = df_ios['customer_phone'].nunique()
number_tasks_ios= df_ios['task_id'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну задачу:", unique_users_ios)
st.write("Количество созданных задач:", number_tasks_ios)

In [22]:
st.write("""## Android""")

In [23]:
unique_users_android = df_andoid['customer_phone'].nunique()
number_tasks_android = df_andoid['task_id'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну задачу:", unique_users_android)
st.write("Количество созданных задач:", number_tasks_android)

In [24]:
st.write("""### Распределение пользователей по платформе""")

In [25]:
platform_distribution = Counter(df['plathorm'])

st.write("Количество созданных задач:", platform_distribution)

# Повторные задачи

In [27]:
st.write("""# Повторные задачи""")

In [28]:
df = df.loc[df['diff'] > '120 seconds']
df_admins_new = df[df['plathorm'] == 'admins']
df_ios_new = df[df['plathorm'] == 'ios']
df_android_new = df[df['plathorm'] == 'android']

## Admins

In [29]:
st.write("""## Admins""")

In [35]:
unique_users_new_df_admins = df_admins_new['customer_phone'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну повторную  задачу:", unique_users_new_df_admins)
st.write('Процент повторных задач в панели администратора: {:.0f}%'
        .format(unique_users_new_df_admins * 100 / unique_users_admins))

## iOS

In [31]:
st.write("""## iOS""")

In [36]:
unique_users_new_df_ios = df_ios_new['customer_phone'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну повторную  задачу:", unique_users_new_df_ios)
st.write('Процент повторных задач iOS: {:.0f}%'
        .format(unique_users_new_df_ios * 100 / unique_users_ios))

## Android

In [33]:
st.write("""## Android""")

In [37]:
unique_users_new_df_android = df_android_new['customer_phone'].nunique()

st.write("Количество пользователей, которые создали хотя бы одну повторную  задачу:", unique_users_new_df_android)
st.write('Процент повторных задач iOS: {:.0f}%'
        .format(unique_users_new_df_android * 100 / unique_users_android))

# Медиана и среднее значение

In [38]:
st.write("""# Медиана и среднее значение""")

## Admins

In [39]:
st.write("""## Admins""")

In [43]:
st.write('Среднее повторной задачи Admins', df_admins_new['diff'].mean())
st.write('Медиана повторной задачи Admins', df_admins_new['diff'].median())

## iOS

In [47]:
st.write("""## iOS""")

In [44]:
st.write('Среднее повторной задачи iOS', df_ios_new['diff'].mean())
st.write('Медиана повторной задачи iOS', df_ios_new['diff'].median())

## Android

In [45]:
st.write("""## Android""")

In [46]:
st.write('Среднее повторной задачи Android', df_android_new['diff'].mean())
st.write('Медиана повторной задачи Android', df_android_new['diff'].median())

# Распределение по дням

In [58]:
st.write("""# Распределение по дням""")

In [48]:
def creation_day(day):
    if int(day.days) <= 2:
        return 'Меньше 2 дней'
    elif 3 <= int(day.days) < 5:
        return 'От 2 до 5 дней'
    elif 5 <= int(day.days) < 10:
        return 'От 5 до 10 дней'
    elif 10 <= int(day.days) < 20:
        return 'От 10 до 20 дней'
    elif 20 <= int(day.days) < 50:
        return 'От 20 до 50 дней'
    else:
        return 'Более 50 дней'

## Admins

In [59]:
st.write("""# Admins""")

In [54]:
st.write('Распределение повторных задач Admins', df_admins_new['diff'].apply(creation_day).value_counts())

## iOS

In [60]:
st.write("""# iOS""")

In [55]:
st.write('Распределение повторных задач iOS', df_ios_new['diff'].apply(creation_day).value_counts())

## Android

In [61]:
st.write("""# Android""")

In [56]:
st.write('Распределение повторных задач Android', df_android_new['diff'].apply(creation_day).value_counts())